In [82]:
import metl
import torch
from biopandas.pdb import PandasPdb
from Bio.SeqUtils import seq1


In [12]:
# model, encoder = metl.get_from_ident("metl-l-2m-3d-gb1")
model = torch.load(open('./model.pt', 'rb'))
encoder = torch.load(open('./encoder.pt', 'rb'))

C:\Users\johng\AppData\Local\Temp\ipykernel_6520\3440960159.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(open('./model.pt', 'rb'))
C:\Users\johng\A

In [17]:
wt = "MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"
pdb_fn = '../pdbs/2qmt_p.pdb'
variants = ["T17P,T54F", "V28L,F51A"]

In [85]:
ppdb = PandasPdb().read_pdb(pdb_fn)
print(ppdb.df.keys())
print(ppdb.df['ATOM'].columns)
groups = ppdb.df['ATOM'].groupby('residue_number')
wt_seq = []
for group_name, group_data in groups:
    wt_seq.append(seq1(group_data.iloc[0]['residue_name']))
''.join(wt_seq)

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])
Index(['record_name', 'atom_number', 'blank_1', 'atom_name', 'alt_loc',
       'residue_name', 'blank_2', 'chain_id', 'residue_number', 'insertion',
       'blank_3', 'x_coord', 'y_coord', 'z_coord', 'occupancy', 'b_factor',
       'blank_4', 'segment_id', 'element_symbol', 'charge', 'line_idx'],
      dtype='object')


True

In [14]:
encoded_variants = encoder.encode_variants(wt, variants)

In [86]:
encoded_variants

array([[11, 14, 20,  9, 10,  8, 10, 12,  6,  9, 17, 10,  9,  6,  4, 17,
        17, 13,  4,  1, 18,  3,  1,  1, 17,  1,  4,  9, 18,  5,  9, 14,
        20,  1, 12,  3, 12,  6, 18,  3,  6,  4, 19, 17, 20,  3,  3,  1,
        17,  9, 17,  5, 17, 18,  5,  4],
       [11, 14, 20,  9, 10,  8, 10, 12,  6,  9, 17, 10,  9,  6,  4, 17,
        17, 17,  4,  1, 18,  3,  1,  1, 17,  1,  4,  9, 10,  5,  9, 14,
        20,  1, 12,  3, 12,  6, 18,  3,  6,  4, 19, 17, 20,  3,  3,  1,
        17,  9, 17,  1, 17, 18, 17,  4]])

In [20]:
with torch.no_grad():
    logits = model(encoded_variants, pdb_fn = pdb_fn)
print(logits)

tensor([[ 0.8411, -0.2675,  0.4626,  0.8375,  0.1142,  0.3604,  0.7205, -0.7480,
         -0.0744,  2.3942,  0.7609,  0.3872, -0.1821,  1.4607, -0.0923,  1.2541,
          0.5510, -0.7746, -0.4496,  0.4191,  0.8662,  0.8771,  0.5245,  1.4875,
          0.3771,  0.5041, -0.3007,  0.1502, -0.4211,  0.3817,  0.5023,  0.2472,
         -0.0427, -0.1571,  0.9614,  0.6088,  1.4072,  0.0447, -0.7701,  1.0510,
          1.2042,  0.5023, -0.9959,  0.7970,  1.3743,  0.8026, -0.4153, -0.1811,
          1.9806,  0.7943, -0.8579,  2.2671, -0.0376,  1.6017, -0.1628],
        [ 0.0143,  1.2544,  0.5280, -0.8936, -0.4795, -0.1708, -0.4251, -0.0408,
         -0.1400, -0.3338, -0.3094, -0.9919,  1.1584,  0.1949, -0.6159, -0.1772,
         -0.3620, -0.2166, -0.4934, -1.4132, -0.4597, -1.3537, -0.6285, -0.9735,
          0.3244,  0.2013,  0.5022,  0.3627, -0.0778,  0.2239,  0.5389,  0.2339,
         -0.6618,  0.0958, -0.3920, -0.5966, -0.7410,  0.0436, -0.2070, -0.9669,
         -1.0585,  0.5389, -1.2097, 